In [ ]:
# Install FastAPI and Uvicorn
!pip install fastapi uvicorn

In [ ]:
#Import the neccessary libraries
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, recall_score, precision_score
import fastapi
from typing import Optional

In [ ]:


_x = [1]
_y = [1]
_prediction = ["True"]
_result = ["True"]
model = DummyClassifier(strategy="constant", constant="True")
model.fit(np.array([_x,_y]).reshape(1, -1), _result)


DummyClassifier(constant='True', strategy='constant')

In [ ]:
# Evaluate the model's performance on test data and return accuracy, recall, and precision
def model_evaluation(y_test, y_pred) :
    print('Trained Model Test Data Accuracy Score :',accuracy_score(y_test, y_pred)*100)
    #testacc=accuracy_score(y_test, y_pred)
    #testrecall=recall_score(y_test, y_pred)
    #testprecision=precision_score(y_test, y_pred)
    #Inserting the pos_label
    testacc = accuracy_score(y_test, y_pred)
    testrecall = recall_score(y_test, y_pred, pos_label='True', zero_division=0)
    testprecision = precision_score(y_test, y_pred, pos_label='True', zero_division=0)
    print(' ')
    print(classification_report(y_test, y_pred))
    return testacc, testrecall, testprecision

In [ ]:
# Create a DataFrame for model evaluation with predictions, correctness, and True Positive detection
def create_eval_df(x, y, result) :
    data = {"x": x, "y": y, "result": result}
    df = pd.DataFrame(data)
    eval_df = df.copy()
    eval_df["predict"] = model.predict(data[x,y])
    eval_df["correct"] = eval_df["predict"] == eval_df["result"]
    eval_df["correctandtrue"] = eval_df["predict"].isin([True]) & eval_df["result"].isin([True])
    return eval_df

In [ ]:
app = fastapi.FastAPI()


@app.get("/predict/")
def predict(id:int = 0 , x:float =0, y:float=0):
    print(id, x, y)
    _x.insert(id, x)
    _y.insert(id, y)
    prediction = model.predict(np.array([x,y]).reshape(1, -1))
    _prediction.insert(id, prediction[0])
    print(prediction[0])
    return dict([("prediction", prediction[0])])

@app.get("/result/")
def result(id:int, result:str):
    print(result)
    _result.insert(id, result)
    return dict([("message", "Copied result for {}".format(id))])

@app.get("/retrain/")
def retrain(last_n:int):
    print ("RETRAINING over last {}".format(last_n))
    #print(_x)
    #print(_y)
   # X = np.rec.fromarrays([_x, _y])[:(-1*last_n)]
   # Y = np.rec.fromarrays([_result])[:(-1*last_n)]

# Select the last 'last_n' data points as input (X) and target (Y) for prediction or testing

    print(f"RETRAINING over last {last_n} samples")
    X = np.array(list(zip(_x[-last_n:], _y[-last_n:])))
    Y = np.array(_result[-last_n:])

    #print(X, Y)
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.40)

    print(X_train)
    print(y_train)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    testacc, testrecall, testprecision = model_evaluation(y_test, y_pred)

    return {"message": "Model retrained", "accuracy": testacc, "recall": testrecall, "precision": testprecision}

@app.get("/new_model/")
def new_model(model:str, last_n:int, strategy:Optional[str] = None, k:Optional[int] = 0):
    print(model, last_n, strategy, k)
    # Initialize K-Nearest Neighbors classifier with the chosen number of neighbors (k)
    if model == "Dummy" :
        model = DummyClassifier(strategy=strategy)
    elif model == "KNN" :
        model = KNeighborsClassifier(n_neighbors=k)
    #model.set_params(model_params)

    message = retrain(last_n=last_n)

    returns = {"message": "Model trained {}".format(model), "train": message}
    print(returns)

    return returns

#@app.get("/evaluate_model/")
#def evaluate_model(last_n:int):
#    return {"message": "Item created", "last_n": last_n}

In [ ]:
#  Generate 50 sample data points where _x and _y are equal to i, and _result alternates between "True" and "False"
for i in range(50):
    _x.append(i)
    _y.append(i)
    _result.append("True" if i % 2 == 0 else "False")

In [ ]:
# Creating a DummyClassifier with stratified strategy for baseline comparison (50 samples)
new_model("Dummy", 50, strategy="stratified")

Dummy 50 stratified 0
RETRAINING over last 50
RETRAINING over last 50 samples
[[33 33]
 [30 30]
 [ 5  5]
 [16 16]
 [15 15]
 [ 1  1]
 [47 47]
 [ 4  4]
 [ 8  8]
 [45 45]
 [41 41]
 [43 43]
 [37 37]
 [ 9  9]
 [44 44]
 [38 38]
 [42 42]
 [48 48]
 [23 23]
 [21 21]
 [31 31]
 [49 49]
 [39 39]
 [12 12]
 [29 29]
 [ 0  0]
 [ 3  3]
 [27 27]
 [14 14]
 [28 28]]
['False' 'True' 'False' 'True' 'False' 'False' 'False' 'True' 'True'
 'False' 'False' 'False' 'False' 'False' 'True' 'True' 'True' 'True'
 'False' 'False' 'False' 'False' 'False' 'True' 'False' 'True' 'False'
 'False' 'True' 'True']
Trained Model Test Data Accuracy Score : 65.0
 
              precision    recall  f1-score   support

       False       0.00      0.00      0.00         7
        True       0.65      1.00      0.79        13

    accuracy                           0.65        20
   macro avg       0.33      0.50      0.39        20
weighted avg       0.42      0.65      0.51        20

{'message': "Model trained DummyClassifier(

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'message': "Model trained DummyClassifier(strategy='stratified')",
 'train': {'message': 'Model retrained',
  'accuracy': 0.65,
  'recall': 1.0,
  'precision': 0.65}}

In [ ]:
#Create and evaluate a KNN model named "KNN" using 50 data points and 4 neighbors

new_model("KNN", 50, k=4)

KNN 50 None 4
RETRAINING over last 50
RETRAINING over last 50 samples
[[ 5  5]
 [19 19]
 [37 37]
 [ 2  2]
 [31 31]
 [48 48]
 [46 46]
 [ 8  8]
 [15 15]
 [45 45]
 [21 21]
 [20 20]
 [36 36]
 [17 17]
 [47 47]
 [18 18]
 [ 4  4]
 [ 1  1]
 [49 49]
 [35 35]
 [24 24]
 [10 10]
 [11 11]
 [32 32]
 [ 7  7]
 [16 16]
 [ 9  9]
 [29 29]
 [41 41]
 [43 43]]
['False' 'False' 'False' 'True' 'False' 'True' 'True' 'True' 'False'
 'False' 'False' 'True' 'True' 'False' 'False' 'True' 'True' 'False'
 'False' 'False' 'True' 'True' 'False' 'True' 'False' 'True' 'False'
 'False' 'False' 'False']
Trained Model Test Data Accuracy Score : 65.0
 
              precision    recall  f1-score   support

       False       0.00      0.00      0.00         7
        True       0.65      1.00      0.79        13

    accuracy                           0.65        20
   macro avg       0.33      0.50      0.39        20
weighted avg       0.42      0.65      0.51        20

{'message': 'Model trained KNeighborsClassifier(n_n

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'message': 'Model trained KNeighborsClassifier(n_neighbors=4)',
 'train': {'message': 'Model retrained',
  'accuracy': 0.65,
  'recall': 1.0,
  'precision': 0.65}}

In [ ]:
#print(predict(1, 1, 1))
#result(1, True)
new_model("Dummy", 50, "stratified")

Dummy 50 stratified 0
RETRAINING over last 50
RETRAINING over last 50 samples
[[49 49]
 [38 38]
 [34 34]
 [17 17]
 [48 48]
 [41 41]
 [21 21]
 [26 26]
 [28 28]
 [ 6  6]
 [18 18]
 [25 25]
 [ 8  8]
 [29 29]
 [20 20]
 [14 14]
 [ 3  3]
 [39 39]
 [10 10]
 [30 30]
 [40 40]
 [12 12]
 [46 46]
 [43 43]
 [37 37]
 [13 13]
 [33 33]
 [ 4  4]
 [ 2  2]
 [ 0  0]]
['False' 'True' 'True' 'False' 'True' 'False' 'False' 'True' 'True' 'True'
 'True' 'False' 'True' 'False' 'True' 'True' 'False' 'False' 'True' 'True'
 'True' 'True' 'True' 'False' 'False' 'False' 'False' 'True' 'True' 'True']
Trained Model Test Data Accuracy Score : 35.0
 
              precision    recall  f1-score   support

       False       0.00      0.00      0.00        13
        True       0.35      1.00      0.52         7

    accuracy                           0.35        20
   macro avg       0.17      0.50      0.26        20
weighted avg       0.12      0.35      0.18        20

{'message': "Model trained DummyClassifier(strateg

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'message': "Model trained DummyClassifier(strategy='stratified')",
 'train': {'message': 'Model retrained',
  'accuracy': 0.35,
  'recall': 1.0,
  'precision': 0.35}}

In [ ]:
@app.get("/items/")
async def read_item(skip: int = 0, limit: int = 10):
    return {"skip": skip, "limit": limit}

In [ ]:
import asyncio
import uvicorn

if __name__ == "__main__":
    config = uvicorn.Config(app)
    server = uvicorn.Server(config)
    loop = asyncio.get_running_loop()
    loop.create_task(server.serve())